In [1]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

Azure-Hack-2024/.env内にAZURE_OPENAI_ENDPOINTとAZURE_OPENAI_API_KEYを置いている前提

In [2]:
# Azure OpenAIのGPT-4を使って文章生成できるかテスト
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)

In [3]:

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

In [4]:
# azure ai search を使って検索できるかテスト
# https://pypi.org/project/azure-search-documents/
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

In [5]:
service_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = os.environ["AZURE_SEARCH_INDEX_NAME"]
key = os.environ["AZURE_SEARCH_API_KEY"]

In [6]:
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

In [7]:
results = search_client.search(search_text="nameがそうなんだ", top=5,logging_enable=True)

In [8]:
for result in results:
    print(result)